In [43]:
import pandas as pd
import numpy as np

import seaborn as sns
%matplotlib inline

PATH_RATINGS = '../data/snap_amazon/video_games/ratings.msg'
PATH_META = '../data/snap_amazon/video_games/meta.msg'
PATH_IMG_EMBS = '../data/snap_amazon/video_games/img_embs.msg'

ratings_col = 'rating'

In [2]:
ratings_df = pd.read_msgpack(PATH_RATINGS)

In [5]:
ratings_df.rating.value_counts()

5.0    709770
4.0    260260
1.0    152840
3.0    124370
2.0     77513
Name: rating, dtype: int64

In [21]:
ratings_unary_df = ratings_df.loc[ratings_df[ratings_col] >= 5.0].drop(ratings_col, axis=1)
ratings_unary_df.to_msgpack('../data/snap_amazon/video_games/ratings_unary.msg')

# User Features

In [40]:
s = ratings_df.groupby('user_id').rating.sum()


In [64]:
prolific_s = np.log1p(s) >= np.log1p(s).median()
prolific_s = prolific_s.astype('category')
prolific_s.name = 'prolific'

In [121]:
user_feats_df = pd.concat([
    prolific_s
], axis=1)

In [122]:
user_feats_df.to_msgpack('../data/snap_amazon/video_games/user_feats.msg')

# Item Features

In [69]:
meta_df = pd.read_msgpack(PATH_META)

In [114]:
def extract_level(x):
    x0 = x[0]
    if len(x0) > 1:
        return x0[1]
    else:
        return x0[0]
    
some_category_s = meta_df.categories.apply(extract_level)

some_category_s = some_category_s.astype('category')
some_category_s.name = 'game_cat'

In [123]:
item_feats_df = pd.concat([
    some_category_s
], axis=1)

In [124]:
item_feats_df.to_msgpack('../data/snap_amazon/video_games/item_feats.msg')